# 책소개
수집된 a_isbn을 통해 책소개(introduction) 수집(책 당 1개 책소개 수집) 
- 사용파일: A_information_etc_utf8.csv
- 생성파일: A_introductions_utf8.csv

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
from selenium import webdriver
import csv
import time
import pandas as pd

In [6]:
# CSV 파일 경로
# 알라딘-총평점-v1.0 작업 이후 A_information_etc 파일로 작업필요
file_path = 'A_information_etc_utf8.csv'

# pandas를 사용하여 CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 'a_isbn' 열의 값들을 중복 없이 리스트로 변환
a_isbn_list = df['a_isbn'].unique().tolist()

# 'item_id'에서 중복된 행만 필터링
duplicated_rows = df[df.duplicated('a_isbn', keep=False)]

# 결과 출력
if len(duplicated_rows) >= 1:
   print(duplicated_rows.sort_values(by='a_isbn'))
else:
    print('중복된 행이 없습니다')

       item_id a_isbn  total_rating
660  309951192    NaN           NaN
823   56868780    NaN           9.6


In [7]:
len(a_isbn_list)

1022

In [8]:
#리스트만들기
introductions = []
introduction_html_list = []

In [9]:
# 요청 헤더
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Cookie": "CheckSameSite4=IsValidSameSiteSet; AladdinUser=UID=1238288727&SID=MZhgpmxTJ5jmKnv24AFjAg%3d%3d; AladdinUS=uz%2bg8Meq6f3tucYt2eBk%2fw%3d%3d&USA=0; _gcl_au=1.1.80453147.1710319532; _BS_GUUID=3jguseZr9W4O9focZdu9DZBb6ksnLWiPfyjHNi8W; _ga=GA1.1.955527986.1710319532; _fwb=191BLSAqxrjXG5K4lL8oJvI.1710319532043; _gcl_aw=GCL.1710380746.Cj0KCQjwncWvBhD_ARIsAEb2HW-2yZmyXOBJ6v__Us_5bPB4TuKw6AbwFUUqqoYHNCsME4FQLGoWATEaAg4qEALw_wcB; _TRK_AUIDA_13987=4dd981c7ce0f287249c7f241078780ea:6; _TRK_ASID_13987=1d0ae7e7a80900d4bd8531578b325f52; refererURL=https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=318773745; wcs_bt=s_1c519d64863a:1710404972; _ga_VKYKBC0ZHH=GS1.1.1710402612.6.1.1710404972.53.0.0",
    "Host": "www.aladin.co.kr",
    "Referer": "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=318773745",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "sec-ch-ua": "\"Chromium\";v=\"122\", \"Not(A:Brand\";v=\"24\", \"Google Chrome\";v=\"122\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\""}

count = 0
for aladinISBN in a_isbn_list[:1]:
    try:
        introduction = None
        url = f"https://www.aladin.co.kr/shop/product/getContents.aspx?ISBN={aladinISBN}&name=Introduce&type=0&date=1"
        introduction_response = requests.get(url, headers=headers)
        #저장
        introduction_soup = BeautifulSoup(introduction_response.text, 'html.parser')
        # 응답 확인    
        # print(response.text)  # 응답 본문 확인    
        introduction = introduction_soup.find("div", class_="Ere_prod_mconts_R").text.strip()
        introduction_dict = {
            'a_isbn': aladinISBN,
            'introduction': introduction
        }   
        # 딕셔너리를 리스트에 추가
        introductions.append(introduction_dict)
        introduction_html = {
        'a_isbn': aladinISBN,
        'html': introduction_soup
        }
        introduction_html_list.append(introduction_html)
    except Exception as e:
        introduction_dict = {
            'a_isbn': aladinISBN,
            'introduction': introduction
        }  
        introductions.append(introduction_dict)
        introduction_html = {
                'a_isbn': aladinISBN,
                'html': None
            }
        introduction_html_list.append(reviews_html)
        continue
    print(count, time.strftime('%Y.%m.%d - %H:%M:%S'), aladinISBN)
    count += 1



0 2024.03.19 - 11:17:06 K752730888


In [16]:
field_names = list(introductions[1].keys())

# 'encoding' 인자를 'utf-8'로 설정
with open('A_introductions_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(introductions)

In [13]:
len(introductions)

201

In [15]:
#csv로 저장
fields = list(introduction_html_list[0].keys())

# 'encoding' 인자를 'utf-8'로 설정
with open('A_introduction_html_list_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(introduction_html_list)